In [ ]:
#REPLACE THE ORIGINAL CONVOLUTION LAYERS WITH THE MULTI-SCALE RESIDUAL ATROUS CONVOLUTION AND 
#ADD MULTI-LEVEL RESIDUAL ATROUS SPATIAL PYRAMID POOLING MODULES BETWEEN THE ENCODING AND DECODING MODULES.

In [ ]:
#model residual atrous convolution  + ASPP
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np,sys
from  scipy.signal import convolve2d
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers import Conv2D,  UpSampling2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, AveragePooling2D, Concatenate, Input, GlobalAveragePooling2D, Reshape, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same", dilation_rate = (2,2))(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(num_filters, 3, padding="same", dilation_rate = (4,4))(x)
    x = BatchNormalization()(x)

    shortcut = Conv2D(num_filters, 1, padding ="same") (inputs)         #RESIDUAL
    shortcut = BatchNormalization()(shortcut)
    
    x= tf.keras.layers.add([shortcut, x])
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    s = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(s)
    return s, p

def ASPP(inputs):
    """ Image Pooling """
    shape = inputs.shape
    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(inputs)
    y1 = Conv2D(512, 1, padding="same", use_bias=False)(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y1)

    """ 1x1 conv """
    y2 = Conv2D(512, 1, padding="same", use_bias=False)(inputs)
    y2 = BatchNormalization()(y2)
    shortcut = Conv2D(512, 1, padding ="same") (inputs)
    shortcut = BatchNormalization()(shortcut)
    x= tf.keras.layers.add([shortcut, y2])
    y2 = Activation("relu")(y2)

    """ 3x3 conv rate=6 """
    y3 = Conv2D(512, 3, padding="same", use_bias=False, dilation_rate=6)(inputs)
    y3 = BatchNormalization()(y3)
    shortcut = Conv2D(512, 3, padding ="same") (inputs)
    shortcut = BatchNormalization()(shortcut)
    x= tf.keras.layers.add([shortcut, y3])
    y3 = Activation("relu")(y3)

    """ 3x3 conv rate=12 """
    y4 = Conv2D(512, 3, padding="same", use_bias=False, dilation_rate=12)(inputs)
    y4 = BatchNormalization()(y4)
    shortcut = Conv2D(512, 3, padding ="same") (inputs)
    shortcut = BatchNormalization()(shortcut)
    x= tf.keras.layers.add([shortcut, y4])
    y4 = Activation("relu")(y4)

    """ 3x3 conv rate=18 """
    y5 = Conv2D(512, 3, padding="same", use_bias=False, dilation_rate=18)(inputs)
    y5 = BatchNormalization()(y5)
    shortcut = Conv2D(512, 3, padding ="same") (inputs)
    shortcut = BatchNormalization()(shortcut)
    x= tf.keras.layers.add([shortcut, y5])
    y5 = Activation("relu")(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])
    y = Conv2D(512, 1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    """input layer"""
    inputs = Input(input_shape)

    """Encoder"""
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    """Bottleneck"""
    b1 = conv_block(p4, 1024)
    A = ASPP(b1)

    """Decoedr"""
    d1 = decoder_block(A, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="MSRAC_MLRASPP_UNET")
    return model

if __name__ == "__main__":
    input_shape = (512, 512, 3)
    model = build_unet(input_shape)
    model.summary()

Model: "MSRAC_MLRASPP_UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                

In [ ]:
#Metrics
from tqdm import tqdm
import imageio
from albumentations import HorizontalFlip, VerticalFlip, ElasticTransform, Transpose, RandomRotate90, GridDistortion, OpticalDistortion, CoarseDropout
from google.colab.patches import cv2_imshow

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "mask", "*.jpg")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory to save files """
    create_dir("files")

    """ Hyperparameters """
    batch_size = 4
    lr = 0.001
    num_epochs = 150
    model_path = os.path.join("/content/drive/MyDrive/files", "lastQ1model.h5")
    csv_path = os.path.join("/content/drive/MyDrive/files", "lastQ1data.csv")

    """ Dataset """
    dataset_path = "/content/drive/MyDrive/finaldata"
    train_path = os.path.join(dataset_path, "train")
    valid_path = os.path.join(dataset_path, "val")

    train_x, train_y = load_data(train_path)
    train_x, train_y = shuffling(train_x, train_y)
    valid_x, valid_y = load_data(valid_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

    train_steps = len(train_x)//batch_size
    valid_setps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1
    if len(valid_x) % batch_size != 0:
        valid_setps += 1

    """ Model """
    model = build_unet((H, W, 3))
    model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
    # model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor = 0.01, patience=5, min_lr= 0.0001, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=150, restore_best_weights=False)
    ]
    history =  model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_setps,
        callbacks=callbacks
    )

Train: 160 - 160
Valid: 20 - 20
Epoch 1/150
40/40 [==============================] - ETA: 0s - loss: 0.4480 - dice_coef: 0.5520 - iou: 0.3892 - recall: 0.3137 - precision: 0.6972
Epoch 1: val_loss improved from inf to 0.83210, saving model to /content/drive/MyDrive/files/lastQ1model.h5
40/40 [==============================] - 210s 4s/step - loss: 0.4480 - dice_coef: 0.5520 - iou: 0.3892 - recall: 0.3137 - precision: 0.6972 - val_loss: 0.8321 - val_dice_coef: 0.1679 - val_iou: 0.0918 - val_recall: 0.2554 - val_precision: 0.2683 - lr: 0.0010
Epoch 2/150
40/40 [==============================] - ETA: 0s - loss: 0.3425 - dice_coef: 0.6575 - iou: 0.4914 - recall: 0.2853 - precision: 0.8717
Epoch 2: val_loss improved from 0.83210 to 0.79905, saving model to /content/drive/MyDrive/files/lastQ1model.h5
40/40 [==============================] - 71s 2s/step - loss: 0.3425 - dice_coef: 0.6575 - iou: 0.4914 - recall: 0.2853 - precision: 0.8717 - val_loss: 0.7990 - val_dice_coef: 0.2010 - val_iou: 0.

In [ ]:
#Evaluation
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score


H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg_")))
    y = sorted(glob(os.path.join(path, "mask", "*.jpg_")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255     # 10 pixel white line to separate the images

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)    #concatinated images
    cv2.imwrite(save_image_path, cat_images)

if __name__ == "__main__":
    """ Save the results in this folder """
    create_dir("results")

    """ Load the model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model("/content/drive/MyDrive/files/lastQ1model.h5")

    """ Load the dataset """
    dataset_path = os.path.join("/content/drive/MyDrive/test")
    test_x, test_y = load_data(dataset_path)

    """ Make the prediction and calculate the metrics values """
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
      """ Extracting name """
      name = x.split("/")[-1].split(".")[0]
            
      #Read the image and mask
      ori_x, x = read_image(x)
      ori_y, y = read_mask(y)

      #Prediction 
      y_pred = model.predict(np.expand_dims(x, axis=0))[0]
      y_pred = y_pred > 0.5   # the threshold to make it 0 or 1 predict
      y_pred = y_pred.astype(np.int32)
      y_pred = np.squeeze(y_pred, axis=-1)

      # Saving the images
      save_image_path = f"/content/drive/MyDrive/finalresult/proimage/{name}.jpg"
      save_results(ori_x, ori_y, y_pred, save_image_path)
   
           
      # Flatten the array
      y = y.flatten()
      y_pred = y_pred.flatten()

      #Calculate the metrics 
      acc_value = accuracy_score(y, y_pred)
      f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
      jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
      recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
      precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
      SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")
    

    #Saving 
    df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("/content/drive/MyDrive/finalresult/proscore.csv")

     

  0%|          | 0/20 [00:00<?, ?it/s]

1/1 [==============================] - 10s 10s/step


  5%|▌         | 1/20 [00:13<04:07, 13.05s/it]

1/1 [==============================] - 8s 8s/step


 10%|█         | 2/20 [00:21<03:07, 10.39s/it]

...

1/1 [==============================] - 8s 8s/step


 95%|█████████▌| 19/20 [03:20<00:13, 13.34s/it]

1/1 [==============================] - 8s 8s/step


100%|██████████| 20/20 [03:31<00:00, 10.59s/it]

Accuracy: 0.41568
F1: 0.23361
Jaccard: 0.13235
Recall: 0.13249
Precision: 0.99241


In [ ]:
#model atrous convolution
import tensorflow as tf
import numpy as np,sys
from  scipy.signal import convolve2d
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same", dilation_rate = (2,2))(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(num_filters, 3, padding="same", dilation_rate = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    s = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(s)
    return s, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, dilation_rate = (2,2), padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    """input layer"""
    inputs = Input(input_shape)

    """Encoder"""
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    """Bottleneck"""
    b1 = conv_block(p4, 1024)

    """Decoedr"""
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="Atrous_UNET")
    return model

if __name__ == "__main__":
    input_shape = (512, 512, 3)
    model = build_unet(input_shape)
    model.summary()